# MIT 87/103

In [2]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Funde

In [3]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Ort.ort_kurz,
           t_Komplex.bef_nr, 
           t_Obj.Individuum, 
           t_Obj.GefEinheit, 
           t_Obj.Typ, 
           t_Obj.Anzahl,
           t_Obj.Gewicht,
           t_Obj.Gr_Clist, 
           t_Obj.Art,
           t_Obj.Fabric 
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) Like '%MIT%')
           AND ((t_Komplex.bef_nr) Like '%87/103%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,ort_kurz,bef_nr,Individuum,GefEinheit,Typ,Anzahl,Gewicht,Gr_Clist,Art,Fabric
0,1226,MIT,87/103,2,,,1,28,70,K,1a
1,1246,MIT,87/103,1,1,Imbonga,1,93,120,K,1a
2,1247,MIT,87/103,7,,,1,47,70,K,1c
3,4421,MIT,87/103,"9, 11-13",,,4,77,,K,1a
4,4422,MIT,87/103,8,,,1,9,,K,1c
5,4423,MIT,87/103,10,,,1,13,,K,1e
6,4424,MIT,87/103,3-4,,,1,3,,K,1a
7,4425,MIT,87/103,6,,,1,4,,K,2b
8,4426,MIT,87/103,4,,,1,3,,K,1c
9,6338,MIT,87/103,5,,BBS (?),1,26,70,K,9a


In [4]:
df_pivot1 = df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot1 = pd.DataFrame(df_pivot1)
# df_pivot1 = df_pivot1.reset_index()
df_pivot1.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot1['\%'] = (df_pivot1['Gewicht']/df_pivot1['Gewicht'].sum()*100).round(1)
df_pivot1['Gewicht'] = (df_pivot1['Gewicht'] / 1000).round(2)
df_pivot1.rename(columns={'Gewicht': 'Gewicht (kg)'}, inplace=True)

df_pivot2 = df.pivot_table(values = 'Anzahl', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot2 = pd.DataFrame(df_pivot2)
df_pivot2.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot2['\%'] = (df_pivot2['Anzahl']/df_pivot2['Anzahl'].sum()*100).round(1)

# Tabellen vereinen:
df_concat = pd.concat([df_pivot2, df_pivot1], axis = 1)
df_concat = df_concat.reset_index()
df_concat.rename(columns={'Art': 'Fundkategorie'}, inplace=True)
# df_concat.to_latex('../output/tabs/9-08_PIK871-1_Funde.tex', index = False, escape = False)
df_concat

,Fundkategorie,Anzahl,\%,Gewicht (kg),\%
0,Keramik,13,100,0.3,100


In [3]:
df['Gewicht'].sum()

303

In [4]:
df.pivot_table(values = 'objID', index = 'Fabric', aggfunc = len)

Fabric
1a    4
1c    3
1e    1
2b    1
9a    1
Name: objID, dtype: int64

### Sonst. Funde aus Mitula

In [9]:
sql = """SELECT
           t_Obj.ObjID, 
           t_ort.ort_name,
           t_Komplex.bef_nr, 
           t_Obj.Typ, 
           t_Obj.Gewicht,
           t_Obj.Gr_Clist, 
           t_Obj.Art,
           t_Obj.Fabric 
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) Like '%MIT%')
           AND ((t_Komplex.bef_nr) Not Like '%87/103%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df['Gewicht'] = df['Gewicht'].convert_objects(convert_numeric=True)
# die Fragezeichen bei Typ einfach löschen
# df['Typ'] = df['Typ'].map(lambda x: x.rstrip(' (?)'))

df.pivot_table(values = 'Gewicht', index = 'bef_nr', columns = 'Typ', aggfunc = sum, margins = True)

Typ,,BBS (?),Bondongo (?),Botendo (?),Imbonga,Imbonga (?),EPE (?),PIKMUN,PIKMUN (?),All
bef_nr,,,,,,,,,,
87/101,395,NaN,NaN,NaN,24,62,NaN,NaN,NaN,481
87/102,984,16,53,195,NaN,NaN,14,120,44,1708
All,1379,16,53,195,24,62,14,120,44,2189


In [10]:
df = df.replace('',np.nan, regex=True)
df_pivot = df.pivot_table(values = 'objID', index = 'Typ', aggfunc = len)
df_pivot.plot(kind='pie', autopct='%.0f %%', cmap = 'Paired', figsize=(5, 5))
plt.ylabel('')